In [1]:
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import openpyxl
import matplotlib.pyplot as plt
%matplotlib inline

### This script followes the next steps:
1. Retrieve unique values included in Amsterdam raw data
2. Calculate total population for each year per country
3. Remove empty entries creating new xlsx in process00
4. Rename the attributes based on DST/UNSD abbreviations storing each new file in process01
5. Rename sum file with names with DST/UNSD abbreviations 

In [2]:
city="ams"
base_dir = os.path.dirname(os.path.dirname(os.path.dirname(os.path.abspath("__file__"))))
# Paths for the Population Data --------------------------------------------------------------
#path to ancillary data folder
ancillary_data_folder_path = base_dir + "/data_prep/{}_Projectdata/AncillaryData".format(city)
ancillary_POPdata_folder_path = base_dir + "/data_prep/{}_Projectdata/PopData".format(city)
image_path= base_dir + "/data_prep/{}_Projectdata/PopData/images".format(city)
years_list=[2006,2008,2010,2012,2014,2016,2018] #1992,1994,1996,1998,2000,2002,2004,

### 1. Get list of unique attributes in all files

In [5]:
years_list=[1992,1994,1996,1998,2000,2002,2004,2006,2008,2010,2012,2014,2016,2018]
file_path = ancillary_POPdata_folder_path + "/rawData/nidi/nidi1992.xlsx"
df = pd.read_excel(file_path, header=0, skiprows=2 )
colList = df.columns.tolist()

for year in years_list:
    pathI = ancillary_POPdata_folder_path + "/rawData/nidi/nidi{}.xlsx".format(year)
    dfI = pd.read_excel(pathI, header=0, skiprows=2 )
    unique = [x for x in dfI.columns if x not in colList and (colList.append(x) or True)]
#print(colList)

['grid_id', 'TOTALPOP', 'CHILDREN', 'STUDENTS', 'MOBILE_ADULTS', 'NOT_MOBILE_ADULTS', 'ELDERLY', 'IMMIGRANTS', 'EU_EFTA_IMMIGRANTS', 'NON_EU_EFTA_IMMIGRANTS', 'BIRTHS', 'DEATHS', 'PEOPLE_WHO_MOVED_OUT_OF_GRID', 'IMMIGRANTS_INTO_NETHERLANDS', 'PEOPLE_WHO_MOVED_INTO_GRID', 'IMMIGRANTS_OUTOF_NETHERLANDS', 'VERHUIS_IN_AMS_OUTOF', 'VERHUIS_IN_AMS_INTO', 'NOT_USED_DWELLINGS', 'LET_OUT_DWELLINGS', 'PRIVATELY_OWNED_DWELLINGS', 'TOTAL_AREA_OF_RESIDENCE', 'NUMBER_OF_DWELLINGS', 'NUMBER_OF_ROOMS', 'Unnamed: 24', 'Aden', 'Afghanistan', 'Albanië', 'Algerije', 'Angola', 'Anguilla', 'Argentinië', 'Aruba', 'Australië', 'Australisch Nieuw-Guinea', "Bahama's", 'Bahrein', 'Bangladesh', 'Barbados', 'Bechuanaland', 'België', 'Belgisch-Congo', 'Benin', 'Bermuda', 'Bolivia', 'Bondsrepubliek Duitsland', 'Botswana', 'Brazilië', 'Brits Oost-Afrika', 'Brits West-Indië', 'Brits-Borneo', 'Britse Antillen', 'Britse Maagdeneilanden', 'Brits-Guyana', 'Brits-Honduras', 'Brits-Indië', 'Brits-Somaliland', 'Brunei', 'Bul

In [6]:
## Save the unique values in excel
select_df = pd.DataFrame(colList, columns=["UniqueAttributes"])
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(ancillary_POPdata_folder_path + "/EXCEL/00UniqueAttributes.xlsx",  index = False,  header=True)
# Convert the dataframe to an XlsxWriter Excel object.
select_df.to_excel(writer, sheet_name='Sheet1')
# Close the Pandas Excel writer and output the Excel file.
writer.save()

### Create new dataframe to store the sums of attributes per year

In [7]:
#Remove the standard columns from the unique Attributes and write file
selectList = ['grid_id','grid_geometry_epsg3035','grid_geometry_epsg28992'] 
select = [x for x in colList if not x in selectList]

In [10]:
frame = pd.DataFrame(columns=select)
year = ['1992','1994','1996','1998','2000','2002','2004','2006','2008','2010','2012','2014','2016','2018']
frame['Year'] = year

In [11]:
nframe= frame.set_index('Year')
print(frame)

   TOTALPOP CHILDREN STUDENTS MOBILE_ADULTS NOT_MOBILE_ADULTS ELDERLY  \
0       NaN      NaN      NaN           NaN               NaN     NaN   
1       NaN      NaN      NaN           NaN               NaN     NaN   
2       NaN      NaN      NaN           NaN               NaN     NaN   
3       NaN      NaN      NaN           NaN               NaN     NaN   
4       NaN      NaN      NaN           NaN               NaN     NaN   
5       NaN      NaN      NaN           NaN               NaN     NaN   
6       NaN      NaN      NaN           NaN               NaN     NaN   
7       NaN      NaN      NaN           NaN               NaN     NaN   
8       NaN      NaN      NaN           NaN               NaN     NaN   
9       NaN      NaN      NaN           NaN               NaN     NaN   
10      NaN      NaN      NaN           NaN               NaN     NaN   
11      NaN      NaN      NaN           NaN               NaN     NaN   
12      NaN      NaN      NaN           NaN        

In [16]:
for year in years_list:
    pathI = ancillary_POPdata_folder_path + "/rawData/nidi/nidi{}.xlsx".format(year)
    dfI = pd.read_excel(pathI, header=0, skiprows=2 )
    for x in select: 
        if x in dfI.columns:
            nframe.at['{}'.format(year), x ] = dfI['{}'.format(x)].sum()
        else:
            nframe.at['{}'.format(year), x ] = 0  

In [17]:
print(nframe.head(5))

     TOTALPOP CHILDREN STUDENTS MOBILE_ADULTS NOT_MOBILE_ADULTS ELDERLY  \
Year                                                                      
1992   719923   139564   154253        191592            134534   99980   
1994   722350   143285   144336        198493            139333   96903   
1996   715063   144418   134375        199987            142936   93347   
1998   727095   148286   132247        206379            149416   90767   
2000   734540   151474   125589        212972            156425   88080   

     IMMIGRANTS EU_EFTA_IMMIGRANTS NON_EU_EFTA_IMMIGRANTS BIRTHS  ...  \
Year                                                              ...   
1992      47397              22608                 148276  12427  ...   
1994      47939              22070                 153522  12681  ...   
1996      52367              19577                 152211  12168  ...   
1998      59441              20818                 160178  13218  ...   
2000      68862              22114  

In [18]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(ancillary_POPdata_folder_path + "/EXCEL/sum.xlsx",  index = False,  header=True)
# Convert the dataframe to an XlsxWriter Excel object.
nframe.to_excel(writer, sheet_name='Sheet1')
# Close the Pandas Excel writer and output the Excel file.
writer.save()

### Remove the empty entries and write new xlsx files in process00

In [19]:
removeCountries = []
for i in nframe.columns:  
    sum = nframe['{}'.format(i)].sum()
    #print(i,sum)
    if sum==0:
        removeCountries.append(i)

In [20]:
removeCountries_df = pd.DataFrame(removeCountries, columns=["EmptyEntries"])
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(ancillary_POPdata_folder_path + "/EXCEL/00EmptyEntries.xlsx",  index = False,  header=True)
# Convert the dataframe to an XlsxWriter Excel object.
removeCountries_df.to_excel(writer, sheet_name='Sheet1')
# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [22]:
for year in years_list:
    pathI = ancillary_POPdata_folder_path + "/rawData/nidi/nidi{}.xlsx".format(year)
    dfI = pd.read_excel(pathI, header=0, skiprows=2)
    #select_new = dfI.copy
    for i in removeCountries:
        if i in dfI.columns:
            dfI.drop('{}'.format(i), inplace=True, axis=1)
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(ancillary_POPdata_folder_path + "/process00/{}.xlsx".format(year),  index = False,  header=True)
    # Convert the dataframe to an XlsxWriter Excel object.
    dfI.to_excel(writer, sheet_name='Sheet1')
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

In [23]:
# Read 2 xlsx files and rmove empty  entries
empty = pd.read_excel(ancillary_POPdata_folder_path + "/EXCEL/00EmptyEntries.xlsx", usecols=['EmptyEntries'])
unique = pd.read_excel(ancillary_POPdata_folder_path + "/EXCEL/00UniqueAttributes.xlsx", usecols=['UniqueAttributes'] )
cond = unique['UniqueAttributes'].isin(empty['EmptyEntries'])
unique.drop(unique[cond].index, inplace = True)

     UniqueAttributes
0             grid_id
1            TOTALPOP
2            CHILDREN
3            STUDENTS
4       MOBILE_ADULTS
..                ...
465           Vanuatu
466  Amerikaans-Samoa
467    Sint Eustatius
468  Tristan Da Cunha
469       Zuid-Soedan

[349 rows x 1 columns]


In [24]:
# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(ancillary_POPdata_folder_path + "/EXCEL/00codes.xlsx",  index = False,  header=True)
# Convert the dataframe to an XlsxWriter Excel object.
unique.to_excel(writer, sheet_name='Sheet1')
# Close the Pandas Excel writer and output the Excel file.
writer.save()

In [31]:
# Read 2 xlsx files and rmove empty  entries
codesNew = pd.read_excel(ancillary_POPdata_folder_path + "/EXCEL/codes.xlsx")
unsd = pd.read_excel(ancillary_POPdata_folder_path + "/EXCEL/unsd.xlsx" )
condNew = unsd['abbr'].isin(codesNew['abbr'])
unsd.drop(unsd[condNew].index, inplace = True)
print(unsd)

     Unnamed: 0                                       country   code abbr  \
17           17                   French Southern Territories  260.0  ATF   
69           69              Bonaire, Sint Eustatius and Saba  535.0  BES   
83           83                              Saint Barthélemy  652.0  BLM   
88           88                     Sint Maarten (Dutch part)  534.0  SXM   
90           90                      Turks and Caicos Islands  796.0  TCA   
103         103                                 Bouvet Island   74.0  BVT   
113         113  South Georgia and the South Sandwich Islands  239.0  SGS   
120         120                     Saint Pierre and Miquelon  666.0  SPM   
143         143                                   Timor-Leste  626.0  TLS   
175         175                                 Åland Islands  248.0  ALA   
176         176                                      Guernsey  831.0  GGY   
177         177                                        Jersey  832.0  JEY   

### 4. Rename the attributes based on DST & ISO abbreviations for each excel file
----- Manual Processing the file of Unique Attributes to create abbreviations -----  

In [3]:
for year in years_list:
    pathI = ancillary_POPdata_folder_path + "/process00/{}.xlsx".format(year)
    dfI = pd.read_excel(pathI, header=0)
    ndf = dfI.iloc[: , 1:-1].drop(dfI.index[0]) #
    print(ndf.head(2))

    code_path = ancillary_POPdata_folder_path + "/EXCEL/codes.xlsx"
    codes = pd.read_excel(code_path, header=0)
    print(codes)

    df1 = ndf.rename(columns=codes.set_index('land_nld')['l10_abbr'])
    print (df1.head(4))
    print("------------------------------ Creating xlsx:{0} ------------------------------".format(year))
    # Create a Pandas Excel writer using XlsxWriter as the engine.
    writer = pd.ExcelWriter(ancillary_POPdata_folder_path + "/process01/{}.xlsx".format(year),  index = False,  header=True)
    # Convert the dataframe to an XlsxWriter Excel object.
    df1.to_excel(writer, sheet_name='Sheet1')
    # Close the Pandas Excel writer and output the Excel file.
    writer.save()

   grid_geometry_epsg28992  grid_geometry_epsg28992   
1     grid_geometry_epsg3035   grid_geometry_epsg3035   
2                    grid_id                  grid_id   
3                        NaN                 TOTALPOP   
4                        yes                      ABW   
..                       ...                      ...   
344                      NaN        NOT_MOBILE_ADULTS   
345                      NaN                  ELDERLY   
346                      NaN               IMMIGRANTS   
347                      NaN            EU_IMMIGRANTS   
348                      NaN         NONEU_IMMIGRANTS   

                     country                       eu  
0    grid_geometry_epsg28992  grid_geometry_epsg28992  
1     grid_geometry_epsg3035   grid_geometry_epsg3035  
2                    grid_id                  grid_id  
3                        NaN                      NaN  
4                      Aruba                       no  
..                       ...          

### 5. Rename the attributes based on DST & ISO abbreviations for the sum file

In [100]:
sum_df = pd.read_excel(ancillary_POPdata_folder_path + "/EXCEL/sum.xlsx", header=0)

#Remove the standard columns from the unique Attributes and write file
removeColList = empty['EmptyEntries'].to_list() 
ldf = [x for x in sum_df.columns if not x in removeColList]
lframe = pd.DataFrame(sum_df, columns=ldf)

code_path = ancillary_POPdata_folder_path + "/EXCEL/codes.xlsx"
codes = pd.read_excel(code_path, header=0)

#Rename columns
sumDF = lframe.rename(columns=codes.set_index('land_nld')['l10_abbr'])

# Create a Pandas Excel writer using XlsxWriter as the engine.
writer = pd.ExcelWriter(ancillary_POPdata_folder_path + "/EXCEL/sumNew.xlsx",  index = False,  header=True)
# Convert the dataframe to an XlsxWriter Excel object.
sumDF.to_excel(writer, sheet_name='Sheet1')
# Close the Pandas Excel writer and output the Excel file.
writer.save()